In [ ]:
import pandas as pd
import numpy as np
import openpyxl
import warnings
import sys
import os

# Get the directory where this notebook is located
notebook_dir = os.path.dirname(os.path.abspath(''))

# Add the PythonPrep directory to the path
pythonprep_dir = os.path.join(os.path.dirname(notebook_dir), 'PythonPrep') if 'PythonPrep' not in notebook_dir else notebook_dir
sys.path.append(pythonprep_dir)

from paths import main_path

warnings.filterwarnings("ignore")


In [2]:
### Start cleaning

In [ ]:
path_input = main_path + "/Democracy/Democracy_Main/MainAnalysis/input"
df = pd.read_excel(f"{path_input}/outcomes/excess_deaths/excess_death.xlsx")


In [5]:
df.isnull().sum()

country          0
iso3             0
year             0
sex              0
age_group        0
type             0
expected.mean    0
acm.mean         0
excess.mean*     0
dtype: int64

In [6]:
df = df.groupby(['country', 'iso3', 'year', 'age_group', 'type']).sum().reset_index()
df

,country,iso3,year,age_group,type,expected.mean,acm.mean,excess.mean*
0,Afghanistan,AFG,2020,0-24,predicted,116769.312350,116815.700725,0.000000
1,Afghanistan,AFG,2020,25-34,predicted,21816.893873,22309.789050,486.058814
2,Afghanistan,AFG,2020,35-44,predicted,16722.716956,18862.090176,2140.793107
3,Afghanistan,AFG,2020,45-54,predicted,18875.539283,22984.078723,4116.545704
4,Afghanistan,AFG,2020,55-64,predicted,22913.073521,29139.920359,6241.251777
...,...,...,...,...,...,...,...,...
3099,Zimbabwe,ZWE,2021,45-54,predicted,12261.636907,15395.214477,3134.535612
3100,Zimbabwe,ZWE,2021,55-64,predicted,11720.729788,14249.083406,2528.988115
3101,Zimbabwe,ZWE,2021,65-74,predicted,15724.931339,18305.133850,2580.564727
3102,Zimbabwe,ZWE,2021,75-84,predicted,10147.670149,11390.478069,1242.787328


In [7]:
df.country.nunique()

194

In [8]:
df1 = df.copy()
df1['total_excess_mean'] = df1.groupby(['country', 'year'])['excess.mean*'].transform('sum')
df1['excess_mean_proportion'] = df1['excess.mean*'] / df1['total_excess_mean']
df1 = df1[['country', 'year', 'age_group', 'excess.mean*', 'total_excess_mean', 'excess_mean_proportion']]
df1

,country,year,age_group,excess.mean*,total_excess_mean,excess_mean_proportion
0,Afghanistan,2020,0-24,0.000000,26953.949035,0.000000
1,Afghanistan,2020,25-34,486.058814,26953.949035,0.018033
2,Afghanistan,2020,35-44,2140.793107,26953.949035,0.079424
3,Afghanistan,2020,45-54,4116.545704,26953.949035,0.152725
4,Afghanistan,2020,55-64,6241.251777,26953.949035,0.231552
...,...,...,...,...,...,...
3099,Zimbabwe,2021,45-54,3134.535612,14392.375175,0.217791
3100,Zimbabwe,2021,55-64,2528.988115,14392.375175,0.175717
3101,Zimbabwe,2021,65-74,2580.564727,14392.375175,0.179301
3102,Zimbabwe,2021,75-84,1242.787328,14392.375175,0.086350


In [9]:
df2 = df1.copy()
df2 = df2.drop(columns=['excess.mean*'])
avg_proportion = df2.groupby(['country', 'age_group'])['excess_mean_proportion'].mean().reset_index()
avg_proportion['year'] = 2022
df2 = pd.concat([df2, avg_proportion], ignore_index=True)
df2 = df2[['country', 'year', 'age_group', 'total_excess_mean', 'excess_mean_proportion']].sort_values(by=['country', 'year', 'age_group']).reset_index(drop=True)
df2 = df2.drop('total_excess_mean', axis=1)
df3 = df2.copy()
df3

,country,year,age_group,excess_mean_proportion
0,Afghanistan,2020,0-24,0.000000
1,Afghanistan,2020,25-34,0.018033
2,Afghanistan,2020,35-44,0.079424
3,Afghanistan,2020,45-54,0.152725
4,Afghanistan,2020,55-64,0.231552
...,...,...,...,...
4651,Zimbabwe,2022,45-54,0.213918
4652,Zimbabwe,2022,55-64,0.325012
4653,Zimbabwe,2022,65-74,0.395461
4654,Zimbabwe,2022,75-84,0.079714


In [40]:
#### Import deaths

In [ ]:
deaths = pd.read_csv(main_path + '/Democracy/PythonData/total-daily-covid-deaths.csv')
deaths = deaths.drop('Total confirmed deaths due to COVID-19', axis=1)
deaths


,Entity,Code,Day,Daily new confirmed deaths due to COVID-19
0,Afghanistan,AFG,2020-01-05,0.0
1,Afghanistan,AFG,2020-01-06,0.0
2,Afghanistan,AFG,2020-01-07,0.0
3,Afghanistan,AFG,2020-01-08,0.0
4,Afghanistan,AFG,2020-01-09,0.0
...,...,...,...,...
418495,Zimbabwe,ZWE,2024-07-31,0.0
418496,Zimbabwe,ZWE,2024-08-01,0.0
418497,Zimbabwe,ZWE,2024-08-02,0.0
418498,Zimbabwe,ZWE,2024-08-03,0.0


In [11]:
deaths.Entity.nunique()

250

In [48]:
deaths['Day'] = pd.to_datetime(deaths['Day'])
deaths1 = deaths[(deaths['Day'].dt.year >= 2020) & (deaths['Day'].dt.year <= 2022)]
deaths1

,Entity,Code,Day,Daily new confirmed deaths due to COVID-19
0,Afghanistan,AFG,2020-01-05,0.0
1,Afghanistan,AFG,2020-01-06,0.0
2,Afghanistan,AFG,2020-01-07,0.0
3,Afghanistan,AFG,2020-01-08,0.0
4,Afghanistan,AFG,2020-01-09,0.0
...,...,...,...,...
417913,Zimbabwe,ZWE,2022-12-27,0.0
417914,Zimbabwe,ZWE,2022-12-28,0.0
417915,Zimbabwe,ZWE,2022-12-29,0.0
417916,Zimbabwe,ZWE,2022-12-30,0.0


In [49]:
deaths1['Day'] = pd.to_datetime(deaths1['Day'])
deaths1.set_index('Day', inplace=True)
yearly_deaths = deaths1.groupby('Entity').resample('Y').sum()
yearly_deaths.reset_index(inplace=True)
yearly_deaths['Year'] = yearly_deaths['Day'].dt.year
yearly_deaths.drop(columns=['Day'], inplace=True)
yearly_deaths


,Entity,Daily new confirmed deaths due to COVID-19,Year
0,Afghanistan,2158.0,2020
1,Afghanistan,5194.0,2021
2,Afghanistan,493.0,2022
3,Africa,62637.0,2020
4,Africa,165121.0,2021
...,...,...,...
745,Zambia,3315.0,2021
746,Zambia,325.0,2022
747,Zimbabwe,341.0,2020
748,Zimbabwe,4544.0,2021


In [ ]:
### Filter country 

In [50]:
data_death = yearly_deaths[yearly_deaths['Entity'].isin(df3['country'].unique())]
data_death

,Entity,Daily new confirmed deaths due to COVID-19,Year
0,Afghanistan,2158.0,2020
1,Afghanistan,5194.0,2021
2,Afghanistan,493.0,2022
6,Albania,1134.0,2020
7,Albania,2053.0,2021
...,...,...,...
745,Zambia,3315.0,2021
746,Zambia,325.0,2022
747,Zimbabwe,341.0,2020
748,Zimbabwe,4544.0,2021


In [51]:
### Merge with age-grouped df

In [54]:
merged_df = pd.merge(df3, data_death, left_on=['country', 'year'], right_on=['Entity', 'Year'])

# merged_df['deaths'] = merged_df['excess_mean_proportion'] * merged_df['Daily new confirmed deaths due to COVID-19']
# merged_df = merged_df.drop(columns=['Entity', 'Year'])
# grouped = merged_df.groupby(['country', 'year'])
# result_dfs = {}
# for (country, year), group in grouped:
#     result_dfs[(country, year)] = group[['age_group', 'deaths']]

merged_df['deaths'] = (merged_df['excess_mean_proportion'] * merged_df['Daily new confirmed deaths due to COVID-19']).round().astype(int)
merged_df['deaths'] = merged_df['deaths'].apply(lambda x: max(x, 0))
final_df = merged_df[['country', 'year', 'age_group', 'deaths']]

In [55]:
final_df

,country,year,age_group,deaths
0,Afghanistan,2020,0-24,0
1,Afghanistan,2020,25-34,39
2,Afghanistan,2020,35-44,171
3,Afghanistan,2020,45-54,330
4,Afghanistan,2020,55-64,500
...,...,...,...,...
4171,Zimbabwe,2022,45-54,160
4172,Zimbabwe,2022,55-64,244
4173,Zimbabwe,2022,65-74,297
4174,Zimbabwe,2022,75-84,60


In [ ]:
### Combine age groups: 0-24, 25-64, 65+

In [59]:
def map_age_group(age_group):
    if age_group == '0-24':
        return '0-24'
    elif age_group in ['25-34', '35-44', '45-54', '55-64']:
        return '25-64'
    else:
        return '65+'

final_df['combined_age_group'] = final_df['age_group'].apply(map_age_group)
combined_df = final_df.groupby(['country', 'year', 'combined_age_group'], as_index=False)['deaths'].sum()
combined_df.rename(columns={'combined_age_group': 'age_group'}, inplace=True)
combined_df

,country,year,age_group,deaths
0,Afghanistan,2020,0-24,0
1,Afghanistan,2020,25-64,1040
2,Afghanistan,2020,65+,1119
3,Afghanistan,2021,0-24,0
4,Afghanistan,2021,25-64,3182
...,...,...,...,...
1561,Zimbabwe,2021,25-64,2887
1562,Zimbabwe,2021,65+,1657
1563,Zimbabwe,2022,0-24,0
1564,Zimbabwe,2022,25-64,404


In [ ]:
### Import population df

In [ ]:
age = pd.read_csv(main_path + '/Democracy/PythonData/population-by-age-group.csv')
age


,Entity,Code,Year,Population - Sex: all - Age: 65+ - Variant: estimates,Population - Sex: all - Age: 25-64 - Variant: estimates,Population - Sex: all - Age: 15-24 - Variant: estimates,Population - Sex: all - Age: 5-14 - Variant: estimates,Population - Sex: all - Age: 0-4 - Variant: estimates
0,Afghanistan,AFG,1950,221587,2881750,1481165,1891648,1300032
1,Afghanistan,AFG,1951,225330,2914332,1503397,1931423,1304861
2,Afghanistan,AFG,1952,228860,2948438,1526378,1971722,1312385
3,Afghanistan,AFG,1953,232109,2982329,1548885,2008840,1324540
4,Afghanistan,AFG,1954,235141,3016447,1570535,2043247,1342583
...,...,...,...,...,...,...,...,...
18939,Zimbabwe,ZWE,2019,539584,5256323,2978535,4240525,2256262
18940,Zimbabwe,ZWE,2020,555982,5338557,3082376,4302963,2246868
18941,Zimbabwe,ZWE,2021,568255,5428327,3185913,4359042,2255554
18942,Zimbabwe,ZWE,2022,579686,5516371,3286295,4408155,2278430


In [94]:
pop = age[age['Year'].isin([2020, 2021, 2022])]
pop

,Entity,Code,Year,Population - Sex: all - Age: 65+ - Variant: estimates,Population - Sex: all - Age: 25-64 - Variant: estimates,Population - Sex: all - Age: 15-24 - Variant: estimates,Population - Sex: all - Age: 5-14 - Variant: estimates,Population - Sex: all - Age: 0-4 - Variant: estimates
70,Afghanistan,AFG,2020,926198,12420582,8444268,10799525,6478381
71,Afghanistan,AFG,2021,941009,12874381,8621571,10997458,6565972
72,Afghanistan,AFG,2022,955674,13234074,8704884,11096249,6587947
144,Africa (UN),NaN,2020,47794788,510015883,266247398,352290520,204459148
145,Africa (UN),NaN,2021,49332071,524755877,273536460,359412182,206702984
...,...,...,...,...,...,...,...,...
18867,Zambia,ZMB,2021,358118,6836665,4000177,5356711,3051903
18868,Zambia,ZMB,2022,374025,7093796,4132434,5451878,3100768
18940,Zimbabwe,ZWE,2020,555982,5338557,3082376,4302963,2246868
18941,Zimbabwe,ZWE,2021,568255,5428327,3185913,4359042,2255554


In [95]:
pop_tot = pop[pop['Entity'].isin(combined_df['country'].unique())].drop('Code', axis=1)
pop_tot

,Entity,Year,Population - Sex: all - Age: 65+ - Variant: estimates,Population - Sex: all - Age: 25-64 - Variant: estimates,Population - Sex: all - Age: 15-24 - Variant: estimates,Population - Sex: all - Age: 5-14 - Variant: estimates,Population - Sex: all - Age: 0-4 - Variant: estimates
70,Afghanistan,2020,926198,12420582,8444268,10799525,6478381
71,Afghanistan,2021,941009,12874381,8621571,10997458,6565972
72,Afghanistan,2022,955674,13234074,8704884,11096249,6587947
218,Albania,2020,423914,1518072,426556,344790,158434
219,Albania,2021,434839,1515015,404689,342066,152842
...,...,...,...,...,...,...,...
18867,Zambia,2021,358118,6836665,4000177,5356711,3051903
18868,Zambia,2022,374025,7093796,4132434,5451878,3100768
18940,Zimbabwe,2020,555982,5338557,3082376,4302963,2246868
18941,Zimbabwe,2021,568255,5428327,3185913,4359042,2255554


In [96]:
pop_tot.rename(columns={
    'Entity': 'country',
    'Year': 'year',
    'Population - Sex: all - Age: 65+ - Variant: estimates': 'population_65_plus',
    'Population - Sex: all - Age: 25-64 - Variant: estimates': 'population_25_to_64',
    'Population - Sex: all - Age: 15-24 - Variant: estimates': 'population_15_to_24',
    'Population - Sex: all - Age: 5-14 - Variant: estimates': 'population_5_to_14',
    'Population - Sex: all - Age: 0-4 - Variant: estimates': 'population_0_to_4'
}, inplace=True)
pop_tot

,country,year,population_65_plus,population_25_to_64,population_15_to_24,population_5_to_14,population_0_to_4
70,Afghanistan,2020,926198,12420582,8444268,10799525,6478381
71,Afghanistan,2021,941009,12874381,8621571,10997458,6565972
72,Afghanistan,2022,955674,13234074,8704884,11096249,6587947
218,Albania,2020,423914,1518072,426556,344790,158434
219,Albania,2021,434839,1515015,404689,342066,152842
...,...,...,...,...,...,...,...
18867,Zambia,2021,358118,6836665,4000177,5356711,3051903
18868,Zambia,2022,374025,7093796,4132434,5451878,3100768
18940,Zimbabwe,2020,555982,5338557,3082376,4302963,2246868
18941,Zimbabwe,2021,568255,5428327,3185913,4359042,2255554


In [98]:
pop_tot['population_0_to_24'] = pop_tot[['population_15_to_24', 'population_5_to_14', 'population_0_to_4']].sum(axis=1)
pop_tot = pop_tot.drop(['population_15_to_24', 'population_5_to_14', 'population_0_to_4'], axis=1)
pop_tot

,country,year,population_65_plus,population_25_to_64,population_0_to_24
70,Afghanistan,2020,926198,12420582,25722174
71,Afghanistan,2021,941009,12874381,26185001
72,Afghanistan,2022,955674,13234074,26389080
218,Albania,2020,423914,1518072,929780
219,Albania,2021,434839,1515015,899597
...,...,...,...,...,...
18867,Zambia,2021,358118,6836665,12408791
18868,Zambia,2022,374025,7093796,12685080
18940,Zimbabwe,2020,555982,5338557,9632207
18941,Zimbabwe,2021,568255,5428327,9800509


In [ ]:
### Add population to main df

In [113]:
df_tot = pd.merge(combined_df, pop_tot, how='left', on=['country', 'year'])
age_group_map = {
    '0-24': 'population_0_to_24',
    '25-64': 'population_25_to_64',
    '65+': 'population_65_plus'
}

df_tot['population'] = df_tot.apply(lambda row: row[age_group_map[row['age_group']]], axis=1)
df_tot = df_tot[['country', 'year', 'age_group', 'deaths', 'population']]
df_tot

,country,year,age_group,deaths,population
0,Afghanistan,2020,0-24,0,25722174
1,Afghanistan,2020,25-64,1040,12420582
2,Afghanistan,2020,65+,1119,926198
3,Afghanistan,2021,0-24,0,26185001
4,Afghanistan,2021,25-64,3182,12874381
...,...,...,...,...,...
1561,Zimbabwe,2021,25-64,2887,5428327
1562,Zimbabwe,2021,65+,1657,568255
1563,Zimbabwe,2022,0-24,0,9972880
1564,Zimbabwe,2022,25-64,404,5516371


In [114]:
# Age specific rate (per 100,000 population)
df_tot['mortality_rate_100k'] = df_tot['deaths'] / df_tot['population'] * 100000
df_tot

,country,year,age_group,deaths,population,mortality_rate_100k
0,Afghanistan,2020,0-24,0,25722174,0.000000
1,Afghanistan,2020,25-64,1040,12420582,8.373199
2,Afghanistan,2020,65+,1119,926198,120.816499
3,Afghanistan,2021,0-24,0,26185001,0.000000
4,Afghanistan,2021,25-64,3182,12874381,24.715751
...,...,...,...,...,...,...
1561,Zimbabwe,2021,25-64,2887,5428327,53.183974
1562,Zimbabwe,2021,65+,1657,568255,291.594443
1563,Zimbabwe,2022,0-24,0,9972880,0.000000
1564,Zimbabwe,2022,25-64,404,5516371,7.323655


In [ ]:
### Age standardized population 
# Source: https://getinthepicture.org/sites/default/files/resources/12.%20Age-standardised%20mortality%20rates.pdf
# Calculation source: https://ourworldindata.org/age-standardization
# https://web.archive.org/web/20230402125741/https://www.statcan.gc.ca/en/dai/btd/asr

In [ ]:
# For country comparison, use world standard pupulation distribution

In [ ]:
pop = pd.read_csv(main_path + '/Democracy/PythonData/population-by-age-group.csv')
pop = pop[pop['Year'].isin([2020, 2021, 2022])]
pop = pop[pop['Entity']=='World']
pop.rename(columns={
    'Entity': 'country',
    'Year': 'year',
    'Population - Sex: all - Age: 65+ - Variant: estimates': 'population_65_plus',
    'Population - Sex: all - Age: 25-64 - Variant: estimates': 'population_25_to_64',
    'Population - Sex: all - Age: 15-24 - Variant: estimates': 'population_15_to_24',
    'Population - Sex: all - Age: 5-14 - Variant: estimates': 'population_5_to_14',
    'Population - Sex: all - Age: 0-4 - Variant: estimates': 'population_0_to_4'
}, inplace=True)
pop['population_0_to_24'] = pop[['population_15_to_24', 'population_5_to_14', 'population_0_to_4']].sum(axis=1)
pop = pop.drop(['population_15_to_24', 'population_5_to_14', 'population_0_to_4'], axis=1).drop('Code', axis=1)
pop['total_population'] = pop[['population_65_plus', 'population_25_to_64', 'population_0_to_24']].sum(axis=1)
pop

,country,year,population_65_plus,population_25_to_64,population_0_to_24,total_population
18718,World,2020,736844945,3882253576,3267423136,7886521657
18719,World,2021,758737437,3921248932,3273956664,7953943033
18720,World,2022,782133168,3959728840,3279020696,8020882704


In [121]:
standardized_df = pd.DataFrame({
    'year': pop['year'],
    'age_standardized_65_plus': pop['population_65_plus'] / pop['total_population'],
    'age_standardized_25_to_64': pop['population_25_to_64'] / pop['total_population'],
    'age_standardized_0_to_24': pop['population_0_to_24'] / pop['total_population']
})
standardized_df

,year,age_standardized_65_plus,age_standardized_25_to_64,age_standardized_0_to_24
18718,2020,0.093431,0.492264,0.414305
18719,2021,0.095391,0.492994,0.411614
18720,2022,0.097512,0.493677,0.408810


In [122]:
### Calculate age-standardized rates
merged_df = pd.merge(df_tot, standardized_df, on='year', how='left')

def get_standardized_rate(row):
    if row['age_group'] == '65+':
        return row['age_standardized_65_plus']
    elif row['age_group'] == '25-64':
        return row['age_standardized_25_to_64']
    elif row['age_group'] == '0-24':
        return row['age_standardized_0_to_24']
    else:
        return None

merged_df['age_standardized_rate'] = merged_df.apply(get_standardized_rate, axis=1)
final_df = merged_df.drop(columns=['age_standardized_65_plus', 'age_standardized_25_to_64', 'age_standardized_0_to_24'])
final_df

,country,year,age_group,deaths,population,mortality_rate_100k,age_standardized_rate
0,Afghanistan,2020,0-24,0,25722174,0.000000,0.414305
1,Afghanistan,2020,25-64,1040,12420582,8.373199,0.492264
2,Afghanistan,2020,65+,1119,926198,120.816499,0.093431
3,Afghanistan,2021,0-24,0,26185001,0.000000,0.411614
4,Afghanistan,2021,25-64,3182,12874381,24.715751,0.492994
...,...,...,...,...,...,...,...
1561,Zimbabwe,2021,25-64,2887,5428327,53.183974,0.492994
1562,Zimbabwe,2021,65+,1657,568255,291.594443,0.095391
1563,Zimbabwe,2022,0-24,0,9972880,0.000000,0.408810
1564,Zimbabwe,2022,25-64,404,5516371,7.323655,0.493677


In [123]:
final_df['age_standard_group'] = final_df['mortality_rate_100k'] * final_df['age_standardized_rate']
final_df

,country,year,age_group,deaths,population,mortality_rate_100k,age_standardized_rate,age_standard_group
0,Afghanistan,2020,0-24,0,25722174,0.000000,0.414305,0.000000
1,Afghanistan,2020,25-64,1040,12420582,8.373199,0.492264,4.121827
2,Afghanistan,2020,65+,1119,926198,120.816499,0.093431,11.287996
3,Afghanistan,2021,0-24,0,26185001,0.000000,0.411614,0.000000
4,Afghanistan,2021,25-64,3182,12874381,24.715751,0.492994,12.184726
...,...,...,...,...,...,...,...,...
1561,Zimbabwe,2021,25-64,2887,5428327,53.183974,0.492994,26.219398
1562,Zimbabwe,2021,65+,1657,568255,291.594443,0.095391,27.815590
1563,Zimbabwe,2022,0-24,0,9972880,0.000000,0.408810,0.000000
1564,Zimbabwe,2022,25-64,404,5516371,7.323655,0.493677,3.615523


In [131]:
# Calculate age-standardized deaths per year
prov_df = final_df.drop(['deaths', 'population', 'mortality_rate_100k', 'age_standardized_rate'], axis=1)
age_standard = prov_df.groupby(['country', 'year']).agg({
    'age_standard_group': 'sum'
}).reset_index()
age_standard['standardized_age'] = age_standard['age_standard_group']
age_standard = age_standard.drop('age_standard_group', axis=1)

In [132]:
age_standard

,country,year,standardized_age
0,Afghanistan,2020,15.409824
1,Afghanistan,2021,32.590780
2,Afghanistan,2022,3.292777
3,Albania,2020,27.283355
4,Albania,2021,49.430813
...,...,...,...
517,Zambia,2021,42.514514
518,Zambia,2022,6.329875
519,Zimbabwe,2020,8.929977
520,Zimbabwe,2021,54.034988


In [ ]:
age_standard.to_csv(f"{path_input}/outcomes/old_outcomes/age_standardized_deaths_copy1.csv", index=False)

In [134]:
age_standard.country.nunique()

174